In [1]:
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split

# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Models
from sklearn.neighbors import NearestNeighbors

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("resources/ml_clean.csv")
print(df.shape)
df.head()

(9507, 21)


,game_id,max_players,max_playtime,min_age,min_players,min_playtime,name,playing_time,category,mechanic,average_rating,users_rated,category_count,mechanic_count,has_expansion,binned_playtime,binned_mechanics,binned_min_age,binned_category,len_description,description_sentiment
0,1,5,240,14,3,240,Die Macher,240,economic,area control / area influence,7.66508,4498,3,5,0,Very Long (121-240 minutes),Strategic Mechanics,Teen (13+),Strategy,222,0.091012
1,2,4,30,12,3,30,Dragonmaster,30,card game,trick-taking,6.60815,478,2,1,0,Short (0-30 minutes),Card & Pool Mechanics,Pre-Teen (10-12),Unknown/Miscellaneous,154,0.055291
2,3,4,60,10,2,30,Samurai,60,abstract strategy,area control / area influence,7.44119,12019,2,4,0,Medium (31-60 minutes),Strategic Mechanics,Pre-Teen (10-12),Strategy,183,-0.056250
3,4,4,60,12,2,60,Tal der Könige,60,ancient,action point allowance system,6.60675,314,1,4,0,Medium (31-60 minutes),Strategic Mechanics,Pre-Teen (10-12),History/War,104,-0.007908
4,5,6,90,12,3,90,Acquire,90,economic,hand management,7.35830,15195,1,3,0,Long (61-120 minutes),Pattern & Design Mechanics,Pre-Teen (10-12),Strategy,191,0.168056


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9507 entries, 0 to 9506
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   game_id                9507 non-null   int64  
 1   max_players            9507 non-null   int64  
 2   max_playtime           9507 non-null   int64  
 3   min_age                9507 non-null   int64  
 4   min_players            9507 non-null   int64  
 5   min_playtime           9507 non-null   int64  
 6   name                   9507 non-null   object 
 7   playing_time           9507 non-null   int64  
 8   category               9507 non-null   object 
 9   mechanic               9507 non-null   object 
 10  average_rating         9507 non-null   float64
 11  users_rated            9507 non-null   int64  
 12  category_count         9507 non-null   int64  
 13  mechanic_count         9507 non-null   int64  
 14  has_expansion          9507 non-null   int64  
 15  binn

In [4]:
# Drop null games
df = df.dropna(how="any").reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9500 entries, 0 to 9499
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   game_id                9500 non-null   int64  
 1   max_players            9500 non-null   int64  
 2   max_playtime           9500 non-null   int64  
 3   min_age                9500 non-null   int64  
 4   min_players            9500 non-null   int64  
 5   min_playtime           9500 non-null   int64  
 6   name                   9500 non-null   object 
 7   playing_time           9500 non-null   int64  
 8   category               9500 non-null   object 
 9   mechanic               9500 non-null   object 
 10  average_rating         9500 non-null   float64
 11  users_rated            9500 non-null   int64  
 12  category_count         9500 non-null   int64  
 13  mechanic_count         9500 non-null   int64  
 14  has_expansion          9500 non-null   int64  
 15  binn

In [5]:
# Select string columns
string_cols = df.select_dtypes(include=['object'])

# Count unique categories for each string column
unique_counts = string_cols.nunique()
unique_counts

name                9365
category              82
mechanic              51
binned_playtime        5
binned_mechanics      10
binned_min_age         5
binned_category       14
dtype: int64

In [6]:
df2=df.copy()

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9500 entries, 0 to 9499
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   game_id                9500 non-null   int64  
 1   max_players            9500 non-null   int64  
 2   max_playtime           9500 non-null   int64  
 3   min_age                9500 non-null   int64  
 4   min_players            9500 non-null   int64  
 5   min_playtime           9500 non-null   int64  
 6   name                   9500 non-null   object 
 7   playing_time           9500 non-null   int64  
 8   category               9500 non-null   object 
 9   mechanic               9500 non-null   object 
 10  average_rating         9500 non-null   float64
 11  users_rated            9500 non-null   int64  
 12  category_count         9500 non-null   int64  
 13  mechanic_count         9500 non-null   int64  
 14  has_expansion          9500 non-null   int64  
 15  binn

In [8]:
df2.columns

Index(['game_id', 'max_players', 'max_playtime', 'min_age', 'min_players',
       'min_playtime', 'name', 'playing_time', 'category', 'mechanic',
       'average_rating', 'users_rated', 'category_count', 'mechanic_count',
       'has_expansion', 'binned_playtime', 'binned_mechanics',
       'binned_min_age', 'binned_category', 'len_description',
       'description_sentiment'],
      dtype='object')

In [9]:
# set columns
meta_cols = ['game_id', 'name']
feature_cols = ['max_players', 'max_playtime', 'min_age', 'min_players',
       'min_playtime', 'playing_time', 'category', 'mechanic',
       'average_rating', 'users_rated', 'category_count', 'mechanic_count',
       'has_expansion', 'binned_playtime', 'binned_mechanics',
       'binned_min_age', 'binned_category', 'len_description',
       'description_sentiment']

In [10]:
# Define Preprocessing Pipelines

# Define preprocessing for numeric features
numeric_features = [ 'max_players', 'max_playtime', 'min_age', 'min_players',
       'min_playtime', 'playing_time', 
       'average_rating', 'users_rated', 'category_count', 'mechanic_count',
       'len_description', 'description_sentiment'] # You can also do this in a loop, select the numeric columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

# Define preprocessing for the binary features
binary_features = ['has_expansion']
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),
    ('label', OrdinalEncoder())])  # Label encode for binary feature

# Define preprocessing for categorical features
categorical_features = ['category', 'mechanic', 'binned_playtime', 'binned_mechanics',
       'binned_min_age', 'binned_category']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing for numeric and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('binary', binary_transformer, binary_features),
        ('cat', categorical_transformer, categorical_features)])

In [11]:
# Correlation Analysis (this is optional and not needed for the ML Experiment)
# It just shows what the data looks like after transformation before training
# We will still declare a full pipeline of preprocessing + training

# Use only preprocessing pipeline to transform the data
preprocessed_X_train = preprocessor.fit_transform(df2).toarray()

# Convert preprocessed data to a DataFrame
# Get the feature names after one-hot encoding
encoded_feature_names = (numeric_features + binary_features +
                         list(preprocessor.transformers_[2][1]['onehot'].get_feature_names_out(categorical_features)))

df_final = pd.DataFrame(preprocessed_X_train, columns=encoded_feature_names)
df_final[meta_cols] = df2.loc[:, meta_cols]
df_final.head()

,max_players,max_playtime,min_age,min_players,min_playtime,playing_time,average_rating,users_rated,category_count,mechanic_count,len_description,description_sentiment,has_expansion,category_abstract strategy,category_action / dexterity,category_adventure,category_age of reason,category_american civil war,category_american indian wars,category_american revolutionary war,category_american west,category_ancient,category_animals,category_arabian,category_aviation / flight,category_bluffing,category_book,category_card game,category_children's game,category_city building,category_civil war,category_civilization,category_collectible components,category_comic book / strip,category_deduction,category_dice,category_economic,category_educational,category_electronic,category_environmental,category_expansion for base-game,category_exploration,category_fantasy,category_farming,category_fighting,category_game system,category_horror,category_humor,category_industry / manufacturing,category_korean war,category_mafia,category_math,category_mature / adult,category_maze,category_medical,category_medieval,category_memory,category_miniatures,category_modern warfare,category_movies / tv / radio theme,category_murder/mystery,category_music,category_mythology,category_napoleonic,category_nautical,category_negotiation,category_novel-based,category_party game,category_pike and shot,category_pirates,category_political,category_post-napoleonic,category_prehistoric,category_print & play,category_puzzle,category_racing,category_real-time,category_religious,category_renaissance,category_science fiction,category_space exploration,category_spies/secret agents,category_sports,category_territory building,category_trains,category_transportation,category_travel,category_trivia,category_video game theme,category_vietnam war,category_wargame,category_word game,category_world war i,category_world war ii,category_zombies,mechanic_acting,mechanic_action / movement programming,mechanic_action point allowance system,mechanic_area control / area influence,mechanic_area enclosure,mechanic_area movement,mechanic_area-impulse,mechanic_auction/bidding,mechanic_betting/wagering,mechanic_campaign / battle card driven,mechanic_card drafting,mechanic_chit-pull system,mechanic_co-operative play,mechanic_commodity speculation,mechanic_crayon rail system,mechanic_deck / pool building,mechanic_dice rolling,mechanic_grid movement,mechanic_hand management,mechanic_hex-and-counter,mechanic_line drawing,mechanic_memory,mechanic_modular board,mechanic_paper-and-pencil,mechanic_partnerships,mechanic_pattern building,mechanic_pattern recognition,mechanic_pick-up and deliver,mechanic_player elimination,mechanic_point to point movement,mechanic_press your luck,mechanic_rock-paper-scissors,mechanic_role playing,mechanic_roll / spin and move,mechanic_route/network building,mechanic_secret unit deployment,mechanic_set collection,mechanic_simulation,mechanic_simultaneous action selection,mechanic_singing,mechanic_stock holding,mechanic_storytelling,mechanic_take that,mechanic_tile placement,mechanic_time track,mechanic_trading,mechanic_trick-taking,mechanic_variable phase order,mechanic_variable player powers,mechanic_voting,mechanic_worker placement,binned_playtime_Extended (241+ minutes),binned_playtime_Long (61-120 minutes),binned_playtime_Medium (31-60 minutes),binned_playtime_Short (0-30 minutes),binned_playtime_Very Long (121-240 minutes),binned_mechanics_Action & Movement Mechanics,binned_mechanics_Card & Pool Mechanics,binned_mechanics_Cooperative & Social Mechanics,binned_mechanics_Game Structure Mechanics,binned_mechanics_Other,binned_mechanics_Others,binned_mechanics_Pattern & Design Mechanics,binned_mechanics_Randomness & Chance Mechanics,binned_mechanics_Strategic Mechanics,binned_mechanics_Unique Mechanics,binned_min_age_Early Childhood (4-6),binned_min_age_Middle Childhood (7-9),binned_min_age_Pre-Teen (10-12),binned_min_age_Teen (13+),binned_min_age_Toddler (0-3),binned_category_C

In [12]:
corrs = df_final.loc[:, [x for x in df_final.columns if x not in meta_cols]].corr()
corrs

,max_players,max_playtime,min_age,min_players,min_playtime,playing_time,average_rating,users_rated,category_count,mechanic_count,len_description,description_sentiment,has_expansion,category_abstract strategy,category_action / dexterity,category_adventure,category_age of reason,category_american civil war,category_american indian wars,category_american revolutionary war,category_american west,category_ancient,category_animals,category_arabian,category_aviation / flight,category_bluffing,category_book,category_card game,category_children's game,category_city building,category_civil war,category_civilization,category_collectible components,category_comic book / strip,category_deduction,category_dice,category_economic,category_educational,category_electronic,category_environmental,category_expansion for base-game,category_exploration,category_fantasy,category_farming,category_fighting,category_game system,category_horror,category_humor,category_industry / manufacturing,category_korean war,category_mafia,category_math,category_mature / adult,category_maze,category_medical,category_medieval,category_memory,category_miniatures,category_modern warfare,category_movies / tv / radio theme,category_murder/mystery,category_music,category_mythology,category_napoleonic,category_nautical,category_negotiation,category_novel-based,category_party game,category_pike and shot,category_pirates,category_political,category_post-napoleonic,category_prehistoric,category_print & play,category_puzzle,category_racing,category_real-time,category_religious,category_renaissance,category_science fiction,category_space exploration,category_spies/secret agents,category_sports,category_territory building,category_trains,category_transportation,category_travel,category_trivia,category_video game theme,category_vietnam war,category_wargame,category_word game,category_world war i,category_world war ii,category_zombies,mechanic_acting,mechanic_action / movement programming,mechanic_action point allowance system,mechanic_area control / area influence,mechanic_area enclosure,mechanic_area movement,mechanic_area-impulse,mechanic_auction/bidding,mechanic_betting/wagering,mechanic_campaign / battle card driven,mechanic_card drafting,mechanic_chit-pull system,mechanic_co-operative play,mechanic_commodity speculation,mechanic_crayon rail system,mechanic_deck / pool building,mechanic_dice rolling,mechanic_grid movement,mechanic_hand management,mechanic_hex-and-counter,mechanic_line drawing,mechanic_memory,mechanic_modular board,mechanic_paper-and-pencil,mechanic_partnerships,mechanic_pattern building,mechanic_pattern recognition,mechanic_pick-up and deliver,mechanic_player elimination,mechanic_point to point movement,mechanic_press your luck,mechanic_rock-paper-scissors,mechanic_role playing,mechanic_roll / spin and move,mechanic_route/network building,mechanic_secret unit deployment,mechanic_set collection,mechanic_simulation,mechanic_simultaneous action selection,mechanic_singing,mechanic_stock holding,mechanic_storytelling,mechanic_take that,mechanic_tile placement,mechanic_time track,mechanic_trading,mechanic_trick-taking,mechanic_variable phase order,mechanic_variable player powers,mechanic_voting,mechanic_worker placement,binned_playtime_Extended (241+ minutes),binned_playtime_Long (61-120 minutes),binned_playtime_Medium (31-60 minutes),binned_playtime_Short (0-30 minutes),binned_playtime_Very Long (121-240 minutes),binned_mechanics_Action & Movement Mechanics,binned_mechanics_Card & Pool Mechanics,binned_mechanics_Cooperative & Social Mechanics,binned_mechanics_Game Structure Mechanics,binned_mechanics_Other,binned_mechanics_Others,binned_mechanics_Pattern & Design Mechanics,binned_mechanics_Randomness & Chance Mechanics,binned_mechanics_Strategic Mechanics,binned_mechanics_Unique Mechanics,binned_min_age_Early Childhood (4-6),binned_min_age_Middle Childhood (7-9),binned_min_age_Pre-Teen (10-12),binned_min_age_Teen (13+),binned_min_age_Toddler (0-3),binned_category_C

In [13]:
# User inputs
gamerec_length = 20
game = "Catan" #Circles


game_id = df2.loc[(df2.name == game) ].game_id.values[0]
game_id

13

In [14]:
# Step 1: Get the data (NOTE: this is the original dataframe, not the one used for the correlations)
X = df2.loc[:, feature_cols]

# Preprocess the data
X_preprocessed = preprocessor.transform(X)

# NO TARGET OR TRAIN TEST SPLIT
X.head()

,max_players,max_playtime,min_age,min_players,min_playtime,playing_time,category,mechanic,average_rating,users_rated,category_count,mechanic_count,has_expansion,binned_playtime,binned_mechanics,binned_min_age,binned_category,len_description,description_sentiment
0,5,240,14,3,240,240,economic,area control / area influence,7.66508,4498,3,5,0,Very Long (121-240 minutes),Strategic Mechanics,Teen (13+),Strategy,222,0.091012
1,4,30,12,3,30,30,card game,trick-taking,6.60815,478,2,1,0,Short (0-30 minutes),Card & Pool Mechanics,Pre-Teen (10-12),Unknown/Miscellaneous,154,0.055291
2,4,60,10,2,30,60,abstract strategy,area control / area influence,7.44119,12019,2,4,0,Medium (31-60 minutes),Strategic Mechanics,Pre-Teen (10-12),Strategy,183,-0.056250
3,4,60,12,2,60,60,ancient,action point allowance system,6.60675,314,1,4,0,Medium (31-60 minutes),Strategic Mechanics,Pre-Teen (10-12),History/War,104,-0.007908
4,6,90,12,3,90,90,economic,hand management,7.35830,15195,1,3,0,Long (61-120 minutes),Pattern & Design Mechanics,Pre-Teen (10-12),Strategy,191,0.168056


In [15]:
# define the number of nearest neighbors to consider
k = gamerec_length

# Initialize the nearest neighbors model
model1 = NearestNeighbors(n_neighbors=k, metric="euclidean")

# Fit the model to the preprocessed data
model1.fit(X_preprocessed)

NearestNeighbors(metric='euclidean', n_neighbors=20)

In [16]:
# Extract the features for the specific gane
game_features = df2.loc[df2.game_id == game_id, feature_cols]
game_features_preprocessed = preprocessor.transform(game_features)

# Find the nearest neighbors
distances, indices = model1.kneighbors(game_features_preprocessed)

# get the game names of the nearest neighbors
games = df2.iloc[indices[0]]
games["distance"] = distances[0]

# filter columns
cols = games.columns

games = games.loc[:, cols]
games = games.sort_values(by = "distance")
games.head(10)

,game_id,max_players,max_playtime,min_age,min_players,min_playtime,name,playing_time,category,mechanic,average_rating,users_rated,category_count,mechanic_count,has_expansion,binned_playtime,binned_mechanics,binned_min_age,binned_category,len_description,description_sentiment,distance
12,13,4,120,10,3,60,Catan,120,negotiation,dice rolling,7.26569,67655,1,5,1,Long (61-120 minutes),Action & Movement Mechanics,Pre-Teen (10-12),Social/Party,457,0.098867,0.000000
4700,30549,4,45,8,2,45,Pandemic,45,medical,action point allowance system,7.67234,62377,1,7,1,Medium (31-60 minutes),Action & Movement Mechanics,Middle Childhood (7-9),Real-life/Simulation,237,0.114088,4.495070
544,822,5,45,8,2,30,Carcassonne,45,city building,area control / area influence,7.43536,67056,3,2,1,Medium (31-60 minutes),Strategic Mechanics,Middle Childhood (7-9),Real-life/Simulation,204,-0.013889,5.010849
5039,36218,4,30,13,2,30,Dominion,30,card game,card drafting,7.69995,55930,2,3,1,Short (0-30 minutes),Card & Pool Mechanics,Teen (13+),Unknown/Miscellaneous,289,0.130288,5.854888
5934,68448,7,30,10,2,30,7 Wonders,30,ancient,card drafting,7.83595,51688,4,5,1,Short (0-30 minutes),Game Structure Mechanics,Pre-Teen (10-12),History/War,252,-0.012574,6.978731
3051,9209,5,60,8,2,30,Ticket to Ride,60,trains,hand management,7.48301,48227,2,3,1,Medium (31-60 minutes),Action & Movement Mechanics,Middle Childhood (7-9),Strategy,229,0.042562,7.698811
4737,31260,5,150,12,1,30,Agricola,150,animals,card drafting,8.05581,48261,3,4,1,Very Long (121-240 minutes),Game Structure Mechanics,Pre-Teen (10-12),Strategy,343,0.082667,8.054047
1540,3076,5,150,12,2,90,Puerto Rico,150,city building,variable phase order,8.09157,47789,3,1,1,Very Long (121-240 minutes),Game Structure Mechanics,Pre-Teen (10-12),Real-life/Simulation,381,-0.017889,8.216691
5345,40692,5,80,8,2,40,Small World,80,fantasy,area control / area influence,7.35735,43144,3,4,1,Long (61-120 minutes),Action & Movement Mechanics,Middle Childhood (7-9),Fantasy/Adventure,250,0.084914,9.019044
1369,2651,6,120,12,2,120,Power Grid,120,economic,auction/bidding,7.94499,42036,2,2,1,Long (61-120 minutes),Action & Movement Mechanics,Pre-Teen (10-12),Strategy,215,0.051769,9.432441


In [17]:
# try other distances 

# define the number of nearest neighbors to consider
k = gamerec_length

# Initialize the nearest neighbors model
model2 = NearestNeighbors(n_neighbors=k, metric="manhattan")

# Fit the model to the preprocessed data
model2.fit(X_preprocessed)

NearestNeighbors(metric='manhattan', n_neighbors=20)

In [18]:
# Extract the features for the specific game
game_features = df2.loc[df2.game_id == game_id, feature_cols]
game_features_preprocessed = preprocessor.transform(game_features)


# Find the nearest neighbors
distances, indices = model2.kneighbors(game_features_preprocessed)

# get the game names of the nearest neighbors
games = df2.iloc[indices[0]]
games["distance"] = distances[0]

# filter columns
cols = games.columns

games = games.loc[:, cols]
games = games.sort_values(by = "distance")
games.head(10)

,game_id,max_players,max_playtime,min_age,min_players,min_playtime,name,playing_time,category,mechanic,average_rating,users_rated,category_count,mechanic_count,has_expansion,binned_playtime,binned_mechanics,binned_min_age,binned_category,len_description,description_sentiment,distance
12,13,4,120,10,3,60,Catan,120,negotiation,dice rolling,7.26569,67655,1,5,1,Long (61-120 minutes),Action & Movement Mechanics,Pre-Teen (10-12),Social/Party,457,0.098867,0.000000
4700,30549,4,45,8,2,45,Pandemic,45,medical,action point allowance system,7.67234,62377,1,7,1,Medium (31-60 minutes),Action & Movement Mechanics,Middle Childhood (7-9),Real-life/Simulation,237,0.114088,17.600574
544,822,5,45,8,2,30,Carcassonne,45,city building,area control / area influence,7.43536,67056,3,2,1,Medium (31-60 minutes),Strategic Mechanics,Middle Childhood (7-9),Real-life/Simulation,204,-0.013889,21.039104
5345,40692,5,80,8,2,40,Small World,80,fantasy,area control / area influence,7.35735,43144,3,4,1,Long (61-120 minutes),Action & Movement Mechanics,Middle Childhood (7-9),Fantasy/Adventure,250,0.084914,22.277628
1369,2651,6,120,12,2,120,Power Grid,120,economic,auction/bidding,7.94499,42036,2,2,1,Long (61-120 minutes),Action & Movement Mechanics,Pre-Teen (10-12),Strategy,215,0.051769,22.467400
5934,68448,7,30,10,2,30,7 Wonders,30,ancient,card drafting,7.83595,51688,4,5,1,Short (0-30 minutes),Game Structure Mechanics,Pre-Teen (10-12),History/War,252,-0.012574,22.556225
5039,36218,4,30,13,2,30,Dominion,30,card game,card drafting,7.69995,55930,2,3,1,Short (0-30 minutes),Card & Pool Mechanics,Teen (13+),Unknown/Miscellaneous,289,0.130288,22.635357
3051,9209,5,60,8,2,30,Ticket to Ride,60,trains,hand management,7.48301,48227,2,3,1,Medium (31-60 minutes),Action & Movement Mechanics,Middle Childhood (7-9),Strategy,229,0.042562,23.232638
4737,31260,5,150,12,1,30,Agricola,150,animals,card drafting,8.05581,48261,3,4,1,Very Long (121-240 minutes),Game Structure Mechanics,Pre-Teen (10-12),Strategy,343,0.082667,24.263938
4953,34635,4,90,10,2,60,Stone Age,90,dice,dice rolling,7.63340,30432,2,3,1,Long (61-120 minutes),Card & Pool Mechanics,Pre-Teen (10-12),Children's/Family,198,0.128114,24.527133


In [19]:
# try other distances

# define the number of nearest neighbors to consider
k = gamerec_length

# Initialize the nearest neighbors model
model3 = NearestNeighbors(n_neighbors=k, metric="cosine")

# Fit the model to the preprocessed data
model3.fit(X_preprocessed)

NearestNeighbors(metric='cosine', n_neighbors=20)

In [20]:
# Extract the features for the specific game
game_features = df2.loc[df2.game_id == game_id, feature_cols]
game_features_preprocessed = preprocessor.transform(game_features)


# Find the nearest neighbors
distances, indices = model3.kneighbors(game_features_preprocessed)

# get the game names of the nearest neighbors
games = df2.iloc[indices[0]]
games["distance"] = distances[0]

# filter columns
cols = games.columns

games = games.loc[:, cols]
games = games.sort_values(by = "distance")
games.head(10)



,game_id,max_players,max_playtime,min_age,min_players,min_playtime,name,playing_time,category,mechanic,average_rating,users_rated,category_count,mechanic_count,has_expansion,binned_playtime,binned_mechanics,binned_min_age,binned_category,len_description,description_sentiment,distance
12,13,4,120,10,3,60,Catan,120,negotiation,dice rolling,7.26569,67655,1,5,1,Long (61-120 minutes),Action & Movement Mechanics,Pre-Teen (10-12),Social/Party,457,0.098867,0.000000
4700,30549,4,45,8,2,45,Pandemic,45,medical,action point allowance system,7.67234,62377,1,7,1,Medium (31-60 minutes),Action & Movement Mechanics,Middle Childhood (7-9),Real-life/Simulation,237,0.114088,0.018657
5039,36218,4,30,13,2,30,Dominion,30,card game,card drafting,7.69995,55930,2,3,1,Short (0-30 minutes),Card & Pool Mechanics,Teen (13+),Unknown/Miscellaneous,289,0.130288,0.022214
5345,40692,5,80,8,2,40,Small World,80,fantasy,area control / area influence,7.35735,43144,3,4,1,Long (61-120 minutes),Action & Movement Mechanics,Middle Childhood (7-9),Fantasy/Adventure,250,0.084914,0.022698
3051,9209,5,60,8,2,30,Ticket to Ride,60,trains,hand management,7.48301,48227,2,3,1,Medium (31-60 minutes),Action & Movement Mechanics,Middle Childhood (7-9),Strategy,229,0.042562,0.023026
544,822,5,45,8,2,30,Carcassonne,45,city building,area control / area influence,7.43536,67056,3,2,1,Medium (31-60 minutes),Strategic Mechanics,Middle Childhood (7-9),Real-life/Simulation,204,-0.013889,0.025112
5934,68448,7,30,10,2,30,7 Wonders,30,ancient,card drafting,7.83595,51688,4,5,1,Short (0-30 minutes),Game Structure Mechanics,Pre-Teen (10-12),History/War,252,-0.012574,0.027004
1369,2651,6,120,12,2,120,Power Grid,120,economic,auction/bidding,7.94499,42036,2,2,1,Long (61-120 minutes),Action & Movement Mechanics,Pre-Teen (10-12),Strategy,215,0.051769,0.027064
3612,14996,5,60,8,2,30,Ticket to Ride: Europe,60,trains,card drafting,7.59192,35535,2,4,1,Medium (31-60 minutes),Action & Movement Mechanics,Middle Childhood (7-9),Strategy,214,0.065083,0.033435
4737,31260,5,150,12,1,30,Agricola,150,animals,card drafting,8.05581,48261,3,4,1,Very Long (121-240 minutes),Game Structure Mechanics,Pre-Teen (10-12),Strategy,343,0.082667,0.034285
